In [40]:
import numpy as np

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup 
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure

import time
import serial

# Estalish a connection with the CD48 board
s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000, timeout = 1) # open serial port; port name will 
                                                            # have to be changed on different computers


# Define a class of objects called Counter. Each counter (0-7) will get a widget checkbox 
# for Port A through Port D   
class Counter(object):
    def __init__(self, portA, portB, portC, portD):
        self.portA = portA
        self.portB = portB
        self.portC = portC
        self.portD = portD
    

widgetHeight = 20   


# Create Label Widgets for each Counter Row. Ex) the 0th row will say "Counter 0:"
counter_labels = []

for i in range(8):
    counter_labels.append(PreText(text='Counter: ' + str(i), width = 80, height = widgetHeight))
    

portLabelWidth = 30

# Create Label widgets for each Port Column
portA = PreText(text='A: ', width = portLabelWidth, height = widgetHeight)
portB = PreText(text='B: ', width = portLabelWidth, height = widgetHeight)
portC = PreText(text='C: ', width = portLabelWidth, height = widgetHeight)
portD = PreText(text='D: ', width = portLabelWidth, height = widgetHeight)

# Create Label widgets for telling the user which counters will watch which ports
# create some blank widgets just to "take up space" so the layout looks nicer
blank = PreText(text='', width = 80, height = widgetHeight)
blank2 = PreText(text='', width = 10, height = widgetHeight)
blank3 = PreText(text='', width = 20, height = widgetHeight)

# Create Set Button to send counter information to CD48 board
set_button = Button(
    label='Set Counters',
    disabled=False,
    button_type='success', # 'success', 'info', 'warning', 'danger' or ''
)




CheckboxWidth = 30

# Create the list counters. Each element of counters is an object of the class Counter.
# This means each element of counters will have 4 widget checkboxes, each referred to as portA - portD
# Later, you can call the value of say counter 4, port D checkbox in this way: counters[4].portD.value
counters = []
# loop through 8 times to create 4 widget checkboxes (portA, portB, portC, portD) 
# for each of counters 0 through 7
for i in range(8):
    counters.append(Counter(CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight)))
    
    
boardSettings = []

boardSettings.append(PreText(text="""Board Settings:""", width = 30, height = widgetHeight))

for i in np.arange(12):
    boardSettings.append(PreText(text=""" """, width = 30, height = widgetHeight))


# Define a function that can be called later. This function sends the 'P' command to the board, 
# then determines whether a given counter is currently "watching" a given port. 
# Then set the checkbox values accordingly (1 --> True, 0 --> False), checkboxes are booleans
def set_checkboxes():
    
    # get the board settings
    s.write('P'.encode())
    time.sleep(0.5)
    settings = s.readlines()
    
    for i in np.arange(len(settings)):
        boardSettings[i+1].text = str(settings[i])
    
    # loop through 8 times to glean port settings for each counter
    for i in range(8):
        # for debugging purposes
        #print('counter = ', i)
        #print(settings[i][2:3])
        #print(settings[i][3:4])
        #print(settings[i][4:5])
        #print(settings[i][5:6])
        
        if settings[i][2:3] == b'0':
            counters[i].portA.active = []
        else:
            counters[i].portA.active = [0]
        
        if settings[i][3:4] == b'0':
            counters[i].portB.active = []
        else:
            counters[i].portB.active = [0]

        if settings[i][4:5] == b'0':
            counters[i].portC.active = []
        else:
            counters[i].portC.active = [0]

        if settings[i][5:6] == b'0':
            counters[i].portD.active = []
        else:
            counters[i].portD.active = [0]

# run the set_checkboxes function to give the checkboxes proper 
# intial values that reflect the current board settings
set_checkboxes()




# for debugging purposes
#print('----------------------------------')
#print('Print Port values:')        
#for i in range(8):
#    print(i, ': ', 
#          counters[i].portA.value, 
#          counters[i].portB.value,
#          counters[i].portC.value,
#          counters[i].portD.value)


# Create info labels that can warn the user if an error is made (cannot clear a counter)
info_labels = []

for i in range(8):
    info_labels.append(PreText(text='', width = 400, height = widgetHeight))

    
# Create one horizontal box for each counter, which includes the port checkboxes
counter_boxes = []

for i in range(8):
    counter_boxes.append(row([counter_labels[i], 
                               counters[i].portA, 
                               counters[i].portB, 
                               counters[i].portC, 
                               counters[i].portD, 
                               blank2,
                               info_labels[i]]))
    




# create a horizontal box with the port column labels
ports_box = row([blank, portA, portB, portC, portD])

# display our row with button and info_sent label
button_row = row([blank3, set_button])

boardSettingsColumn = column([boardSettings[0],
                              boardSettings[1],
                              boardSettings[2],
                              boardSettings[3],
                              boardSettings[4],
                              boardSettings[5],
                              boardSettings[6],
                              boardSettings[7],
                              boardSettings[8],
                              boardSettings[9],
                              boardSettings[10],
                              boardSettings[11],
                              boardSettings[12]])

# create a vertical box which contains all the horizontal boxes above
counter_interface = row([column([ports_box, 
                          counter_boxes[0], 
                          counter_boxes[1], 
                          counter_boxes[2], 
                          counter_boxes[3],
                          counter_boxes[4],
                          counter_boxes[5],
                          counter_boxes[6],
                          counter_boxes[7],
                          button_row]), boardSettingsColumn])


# define a function that will be run every time the Set Counters button is clicked
def send_info():
    
    # loop through 8 times, one for each counter
    for i in range(8):
        # set the values A-D to 0
        A = 0
        B = 0
        C = 0
        D = 0
        # clear the warning label
        info_labels[i].text = ''
        # check the checkbox values of portA - portD, and change respective values as needed
        if counters[i].portA.active == [0]:
            A = 1
        if counters[i].portB.active == [0]:
            B = 1
        if counters[i].portC.active == [0]:
            C = 1
        if counters[i].portD.active == [0]:
            D = 1
            
        # if the sum of A - D is at least 1 (aka at least one port box is checked/True)
        # then write the counter setting command to the board
        if A + B + C + D > 0:
            counting_command = 'S'+ str(i) + str(int(A)) + str(int(B)) + str(int(C)) + str(int(D)) + '\n'
            s.write(counting_command.encode())
        # else (if the sum of A - D is 0), warn the user that this is not permitted. Do not send command to board
        else:
            info_labels[i].text = 'Counter ' + str(i) + ' must watch at least 1 port.'
    
    # Let the user know the counter information was sent
    #info_sent.text = 'Counter Information sent to board.'
    #time.sleep(2.0)
    #info_sent.text = ''
    
    # Find out the current board settings and print them for the user
    #s.write('P'.encode())
    #time.sleep(0.5)
    #settings = s.readlines()
    
    #print("Board Settings:")
    #for i in range(len(settings)):
    #    print(settings[i])
      
    # run the set_checkboxes function to ensure that the values of the checkboxes 
    # reflect the current board settings
    set_checkboxes()


    
# call the send_info function when the button is clicked.
set_button.on_click(send_info)


# Add plot and widgets to current document
curdoc().add_root(counter_interface)
curdoc().title = "Set Counters"